# imports

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import lognorm, nbinom, gamma
from datetime import timedelta
import psycopg2

In [66]:
pd.options.display.float_format = '{:.6f}'.format

In [2]:
conn_params = {
    "host" : "srvdados",
    "database" : "postgres",
    "user":"compras",
    "password": "pecist@compr@s2024"
}

In [3]:
ve = """
    SELECT 
    pp.cod_pro,
    pp.cd_loja,
    pp.dt_emissao::date AS dt_emissao,
    SUM(pp.qtde_ven) AS vendas
FROM "D-1".prod_ped pp
JOIN "D-1".cliente c 
    ON pp.codcli = c.codcli
WHERE 
    pp.tipped = 'V'
    AND pp.dt_emissao >= DATE '2019-01-01'
    and pp.dt_emissao < DATE '2026-01-01'
    AND pp.cd_loja NOT IN ('01','08','09')
    and c.codcli not in ('13996','16100','18400','20000','23000','02608','24000','00270','20690','20691','20692','20693','23011','99999','88888','21097')
    and pp.codvde not in ('0100','0001','0006','2319')
    AND c.codarea <> '112'
    AND c.codcid <> '0501'
GROUP BY
    pp.cod_pro,
    pp.cd_loja,
    pp.dt_emissao::date;
"""

In [4]:
dv = """
   SELECT 
    pe.cd_produto AS cod_pro,
    pe.cd_loja,
    e.dt_emissao::date AS dt_emissao,
    SUM(pe.qt_devolve) AS devolucoes
FROM "D-1".prod_ent pe
JOIN "D-1".entrada e
    ON e.cd_loja = pe.cd_loja
    AND e.sg_serie = pe.sg_serie
    AND e.nu_nota = pe.nu_nota
JOIN "D-1".cliente cli
    ON cli.codcli = pe.cd_cliente
WHERE
    e.dt_emissao >= DATE '2019-01-01'
    and e.dt_emissao < DATE '2026-01-01'
    AND e.in_cancela = 'N'
    AND e.in_clifor = 'C'
    AND UPPER(e.nfeenvstat) NOT LIKE '%DENEG%'
    AND pe.cd_cfop NOT IN ('1949', '2949', '1603')
    AND cli.codcli NOT IN ('99999','88888','21097')
    AND cli.codcid <> '0501'
    AND cli.codarea <> '112'
    AND pe.cd_loja NOT IN ('01','08','09')
GROUP BY
    pe.cd_produto,
    pe.cd_loja,
    e.dt_emissao::date;
"""

In [5]:
query3 = """
select pp.cd_produto AS cod_pro, pro.dt_virada, pro.dt_valida 
from"D-1".prod_pro pp
join "D-1".promocao pro on pp.sq_promoca = pro.sq_promoca and pp.cd_tploja = pro.cd_tploja
where pro.cd_tploja = '01'
"""

In [6]:
with psycopg2.connect(**conn_params) as conn:
    promo = pd.read_sql(query3, conn)

C:\Users\priscila.marques\AppData\Local\Temp\ipykernel_20868\1441821859.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  promo = pd.read_sql(query3, conn)


In [7]:
with psycopg2.connect(**conn_params) as conn:
    ven = pd.read_sql(ve, conn)

C:\Users\priscila.marques\AppData\Local\Temp\ipykernel_20868\2245771844.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  ven = pd.read_sql(ve, conn)


In [8]:
with psycopg2.connect(**conn_params) as conn:
    dev = pd.read_sql(dv, conn)

C:\Users\priscila.marques\AppData\Local\Temp\ipykernel_20868\2747580985.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dev = pd.read_sql(dv, conn)


In [9]:
dev

,cod_pro,cd_loja,dt_emissao,devolucoes
0,032067,06,2023-07-26,2.0
1,011931,04,2021-06-09,1.0
2,016343,06,2024-01-26,1.0
3,051431,03,2025-10-31,1.0
4,021772,06,2024-09-20,2.0
...,...,...,...,...
294355,029383,05,2023-07-07,1.0
294356,036176,06,2024-08-26,1.0
294357,002566,05,2023-03-07,1.0
294358,057303,05,2023-11-08,1.0


In [10]:
gaso = pd.read_excel("gasolina.xlsx")

In [11]:
gaso

,MÊS,ANO,PRODUTO,REGIÃO,ESTADO,MUNICÍPIO,NÚMERO DE POSTOS PESQUISADOS,UNIDADE DE MEDIDA,PREÇO MÉDIO REVENDA,DESVIO PADRÃO REVENDA,PREÇO MÍNIMO REVENDA,PREÇO MÁXIMO REVENDA,MARGEM MÉDIA REVENDA,COEF DE VARIAÇÃO REVENDA,PREÇO MÉDIO DISTRIBUIÇÃO,DESVIO PADRÃO DISTRIBUIÇÃO,PREÇO MÍNIMO DISTRIBUIÇÃO,PREÇO MÁXIMO DISTRIBUIÇÃO,COEF DE VARIAÇÃO DISTRIBUIÇÃO
0,2019-01-01,2019,ETANOL HIDRATADO,CENTRO OESTE,DISTRITO FEDERAL,BRASILIA,219,R$/l,3.267,0.113,2.899,3.609,0.32,0.035,2.947,0.141,2.214,3.15,0.048
1,2019-01-01,2019,GASOLINA COMUM,CENTRO OESTE,DISTRITO FEDERAL,BRASILIA,235,R$/l,4.159,0.191,3.789,4.889,0.434,0.046,3.725,0.071,3.61,3.906,0.019
2,2019-01-01,2019,OLEO DIESEL,CENTRO OESTE,DISTRITO FEDERAL,BRASILIA,127,R$/l,3.665,0.161,3.370,3.899,0.467,0.044,3.198,0.111,2.983,3.463,0.035
3,2019-01-01,2019,OLEO DIESEL S10,CENTRO OESTE,DISTRITO FEDERAL,BRASILIA,174,R$/l,3.787,0.144,3.469,3.999,0.468,0.038,3.32,0.123,3.038,3.515,0.037
4,2019-02-01,2019,ETANOL HIDRATADO,CENTRO OESTE,DISTRITO FEDERAL,BRASILIA,179,R$/l,3.173,0.137,2.879,3.499,0.366,0.043,2.806,0.173,2.16,3.096,0.062
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
385,2025-11-01,2025,ETANOL HIDRATADO,CENTRO OESTE,DISTRITO FEDERAL,BRASILIA,87,R$/l,4.590,0.123,4.260,4.780,-,0.027,-,-,-,-,-
386,2025-11-01,2025,GASOLINA ADITIVADA,CENTRO OESTE,DISTRITO FEDERAL,BRASILIA,179,R$/l,6.390,0.144,5.850,6.780,-,0.023,-,-,-,-,-
387,2025-11-01,2025,GASOLINA COMUM,CENTRO OESTE,DISTRITO FEDERAL,BRASILIA,199,R$/l,6.350,0.123,5.850,6.480,-,0.019,-,-,-,-,-
388,2025-11-01,2025,OLEO DIESEL,CENTRO OESTE,DISTRITO FEDERAL,BRASILIA,80,R$/l,5.980,0.119,5.830,6.280,-,0.020,-,-,-,-,-


In [12]:
datas = pd.read_excel("calendario.xlsx")
datas

,DATA,TIPO_DIA,DIA_SEMANA
0,2019-01-01,FERIADO,Terça-feira
1,2019-01-02,POS FERIADO,Quarta-feira
2,2019-01-03,FERIAS,Quinta-feira
3,2019-01-04,FERIAS,Sexta-feira
4,2019-01-05,FERIAS,Sábado
...,...,...,...
2552,2025-12-27,FERIAS,Sábado
2553,2025-12-28,FERIAS,Domingo
2554,2025-12-29,FERIAS,Segunda-feira
2555,2025-12-30,FERIAS,Terça-feira


# tratamento

In [15]:
ven['dt_emissao'] = pd.to_datetime(ven['dt_emissao'])
dev['dt_emissao'] = pd.to_datetime(dev['dt_emissao'])

ven['vendas'] = ven['vendas'].astype(float)
dev['devolucoes'] = dev['devolucoes'].astype(float)

In [16]:
ven_agg = (
    ven
    .groupby(['cod_pro', 'cd_loja', 'dt_emissao'], as_index=False)
    .agg({'vendas': 'sum'})
)

dev_agg = (
    dev
    .groupby(['cod_pro', 'cd_loja', 'dt_emissao'], as_index=False)
    .agg({'devolucoes': 'sum'})
)

In [17]:
base = ven_agg.merge(
    dev_agg,
    on=['cod_pro', 'cd_loja', 'dt_emissao'],
    how='left'
)

base['devolucoes'] = base['devolucoes'].fillna(0)
base['vendas_liquidas'] = base['vendas'] - base['devolucoes']

In [18]:
base

,cod_pro,cd_loja,dt_emissao,vendas,devolucoes,vendas_liquidas
0,000028,03,2021-06-07,1.0,0.0,1.0
1,000028,03,2022-08-26,2.0,0.0,2.0
2,000028,03,2022-09-16,1.0,0.0,1.0
3,000028,03,2023-04-06,1.0,0.0,1.0
4,000028,03,2023-10-26,1.0,0.0,1.0
...,...,...,...,...,...,...
5241008,111551,05,2025-12-26,1.0,0.0,1.0
5241009,111551,06,2025-12-26,1.0,0.0,1.0
5241010,111552,04,2025-12-23,1.0,0.0,1.0
5241011,111552,05,2025-12-23,3.0,0.0,3.0


In [19]:
gaso['MÊS'] = pd.to_datetime(gaso['MÊS'])
inicio = pd.to_datetime('2019-01-01')
fim = pd.to_datetime('2025-12-31')

gaso_gc = gaso[
    (gaso['MÊS'] >= inicio) &
    (gaso['MÊS'] <= fim)
].copy()

In [20]:
gaso_gc = gaso[
    gaso['PRODUTO'] == 'GASOLINA COMUM'
].copy()

In [21]:
gaso_gc

,MÊS,ANO,PRODUTO,REGIÃO,ESTADO,MUNICÍPIO,NÚMERO DE POSTOS PESQUISADOS,UNIDADE DE MEDIDA,PREÇO MÉDIO REVENDA,DESVIO PADRÃO REVENDA,PREÇO MÍNIMO REVENDA,PREÇO MÁXIMO REVENDA,MARGEM MÉDIA REVENDA,COEF DE VARIAÇÃO REVENDA,PREÇO MÉDIO DISTRIBUIÇÃO,DESVIO PADRÃO DISTRIBUIÇÃO,PREÇO MÍNIMO DISTRIBUIÇÃO,PREÇO MÁXIMO DISTRIBUIÇÃO,COEF DE VARIAÇÃO DISTRIBUIÇÃO
1,2019-01-01,2019,GASOLINA COMUM,CENTRO OESTE,DISTRITO FEDERAL,BRASILIA,235,R$/l,4.159,0.191,3.789,4.889,0.434,0.046,3.725,0.071,3.61,3.906,0.019
5,2019-02-01,2019,GASOLINA COMUM,CENTRO OESTE,DISTRITO FEDERAL,BRASILIA,188,R$/l,4.075,0.169,3.689,4.679,0.398,0.041,3.677,0.087,3.52,3.9,0.024
9,2019-03-01,2019,GASOLINA COMUM,CENTRO OESTE,DISTRITO FEDERAL,BRASILIA,188,R$/l,4.251,0.139,3.899,4.670,0.361,0.033,3.89,0.073,3.728,4.067,0.019
13,2019-04-01,2019,GASOLINA COMUM,CENTRO OESTE,DISTRITO FEDERAL,BRASILIA,235,R$/l,4.374,0.119,3.999,4.699,0.318,0.027,4.056,0.1,3.81,4.244,0.025
17,2019-05-01,2019,GASOLINA COMUM,CENTRO OESTE,DISTRITO FEDERAL,BRASILIA,188,R$/l,4.473,0.128,4.199,4.799,0.313,0.029,4.16,0.056,4.014,4.292,0.013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
367,2025-07-01,2025,GASOLINA COMUM,CENTRO OESTE,DISTRITO FEDERAL,BRASILIA,224,R$/l,6.440,0.158,5.780,6.890,-,0.025,-,-,-,-,-
372,2025-08-01,2025,GASOLINA COMUM,CENTRO OESTE,DISTRITO FEDERAL,BRASILIA,198,R$/l,6.480,0.126,5.890,6.590,-,0.019,-,-,-,-,-
377,2025-09-01,2025,GASOLINA COMUM,CENTRO OESTE,DISTRITO FEDERAL,BRASILIA,248,R$/l,6.360,0.215,5.770,6.580,-,0.034,-,-,-,-,-
382,2025-10-01,2025,GASOLINA COMUM,CENTRO OESTE,DISTRITO FEDERAL,BRASILIA,200,R$/l,6.310,0.201,5.680,6.490,-,0.032,-,-,-,-,-


In [22]:
gaso_gc = gaso_gc[[
    'MÊS',
    'PREÇO MÉDIO REVENDA'
]].rename(columns={
    'PREÇO MÉDIO REVENDA': 'preco_gasolina'
})

In [23]:
base['MÊS'] = base['dt_emissao'].dt.to_period('M').dt.to_timestamp()

base = base.merge(
    gaso_gc,
    on='MÊS',
    how='left'
)

In [24]:
base['dt_emissao'] = pd.to_datetime(base['dt_emissao'])
datas['DATA'] = pd.to_datetime(datas['DATA'])

In [25]:
base = base.merge(
    datas[['DATA', 'TIPO_DIA', 'DIA_SEMANA']],
    left_on='dt_emissao',
    right_on='DATA',
    how='left'
)

In [26]:
base = base.drop(columns=['DATA'])

In [27]:
promo['dt_virada'] = pd.to_datetime(promo['dt_virada'])
promo['dt_valida'] = pd.to_datetime(promo['dt_valida'])

base['dt_emissao'] = pd.to_datetime(base['dt_emissao'])

In [28]:
base_promo = base.merge(
    promo,
    on='cod_pro',
    how='left'
)

In [29]:
base_promo['em_promocao'] = (
    (base_promo['dt_emissao'] >= base_promo['dt_virada']) &
    (base_promo['dt_emissao'] <= base_promo['dt_valida'])
).astype(int)

In [30]:
base = (
    base_promo
    .groupby(
        ['cod_pro','cd_loja','dt_emissao'],
        as_index=False
    )
    .agg({
        'vendas': 'first',
        'devolucoes': 'first',
        'vendas_liquidas': 'first',
        'MÊS': 'first',
        'preco_gasolina': 'first',
        'TIPO_DIA': 'first',
        'DIA_SEMANA': 'first',
        'em_promocao': 'max'
    })
)

base

,cod_pro,cd_loja,dt_emissao,vendas,devolucoes,vendas_liquidas,MÊS,preco_gasolina,TIPO_DIA,DIA_SEMANA,em_promocao
0,000028,03,2021-06-07,1.0,0.0,1.0,2021-06-01,5.751,DIA COMUM,Segunda-feira,0
1,000028,03,2022-08-26,2.0,0.0,2.0,2022-08-01,5.340,DIA COMUM,Sexta-feira,0
2,000028,03,2022-09-16,1.0,0.0,1.0,2022-09-01,4.890,DIA COMUM,Sexta-feira,0
3,000028,03,2023-04-06,1.0,0.0,1.0,2023-04-01,5.580,PRE FERIADO,Quinta-feira,0
4,000028,03,2023-10-26,1.0,0.0,1.0,2023-10-01,5.610,DIA COMUM,Quinta-feira,0
...,...,...,...,...,...,...,...,...,...,...,...
5241008,111551,05,2025-12-26,1.0,0.0,1.0,2025-12-01,NaN,POS FERIADO,Sexta-feira,0
5241009,111551,06,2025-12-26,1.0,0.0,1.0,2025-12-01,NaN,POS FERIADO,Sexta-feira,0
5241010,111552,04,2025-12-23,1.0,0.0,1.0,2025-12-01,NaN,FERIAS,Terça-feira,0
5241011,111552,05,2025-12-23,3.0,0.0,3.0,2025-12-01,NaN,FERIAS,Terça-feira,0


In [51]:
base['ano'] = base['dt_emissao'].dt.year

vendas_anuais = (
    base
    .groupby(['cod_pro', 'ano'])['vendas_liquidas']
    .sum()
    .reset_index()
)

In [53]:
media_anual_produto = (
    vendas_anuais
    .groupby('cod_pro')['vendas_liquidas']
    .mean()
    .reset_index(name='media_anual')
)

produtos_validos = media_anual_produto.loc[
    media_anual_produto['media_anual'] > 108,
    'cod_pro'
]

base = base[base['cod_pro'].isin(produtos_validos)].copy()

In [54]:
base

,cod_pro,cd_loja,dt_emissao,vendas,devolucoes,vendas_liquidas,MÊS,preco_gasolina,TIPO_DIA,DIA_SEMANA,em_promocao,ano
317,000045,03,2019-12-06,10.0,0.0,10.0,2019-12-01,4.513,DIA COMUM,Sexta-feira,0,2019
318,000045,03,2019-12-13,30.0,0.0,30.0,2019-12-01,4.513,DIA COMUM,Sexta-feira,0,2019
319,000045,03,2020-01-07,10.0,0.0,10.0,2020-01-01,4.500,FERIAS,Terça-feira,0,2020
320,000045,03,2020-01-27,10.0,0.0,10.0,2020-01-01,4.500,FERIAS,Segunda-feira,0,2020
321,000045,03,2020-01-28,30.0,0.0,30.0,2020-01-01,4.500,FERIAS,Terça-feira,0,2020
...,...,...,...,...,...,...,...,...,...,...,...,...
5240960,111239,07,2025-12-10,24.0,0.0,24.0,2025-12-01,NaN,DIA COMUM,Quarta-feira,0,2025
5240961,111239,07,2025-12-12,24.0,24.0,0.0,2025-12-01,NaN,DIA COMUM,Sexta-feira,1,2025
5240962,111239,07,2025-12-13,6.0,0.0,6.0,2025-12-01,NaN,DIA COMUM,Sábado,1,2025
5240963,111239,07,2025-12-19,2.0,0.0,2.0,2025-12-01,NaN,FERIAS,Sexta-feira,0,2025


In [56]:
media_tipo_dia = (
    base
    .groupby(['cd_loja', 'TIPO_DIA'])['vendas_liquidas']
    .mean()
    .reset_index()
)

media_tipo_dia

,cd_loja,TIPO_DIA,vendas_liquidas
0,03,DIA COMUM,4.024250
1,03,FERIADO,3.204301
2,03,FERIAS,4.012740
3,03,POS FERIADO,3.853200
4,03,PRE FERIADO,4.188509
5,04,DIA COMUM,5.830514
6,04,FERIADO,6.014019
7,04,FERIAS,6.020204
8,04,POS FERIADO,5.835653
9,04,PRE FERIADO,5.933302


In [57]:
media_dia_semana = (
    base
    .groupby(['cd_loja', 'DIA_SEMANA'])['vendas_liquidas']
    .mean()
    .reset_index()
)

media_dia_semana

,cd_loja,DIA_SEMANA,vendas_liquidas
0,03,Quarta-feira,4.070138
1,03,Quinta-feira,4.161624
2,03,Segunda-feira,3.905540
3,03,Sexta-feira,4.174647
4,03,Sábado,3.084980
5,03,Terça-feira,4.141863
6,04,Domingo,3.387500
7,04,Quarta-feira,6.151635
8,04,Quinta-feira,6.166355
9,04,Segunda-feira,5.781384


In [58]:
from scipy.stats import kruskal

resultado_tipo_dia = []

for loja, df_loja in base.groupby('cd_loja'):
    grupos = [
        g['vendas_liquidas'].values
        for _, g in df_loja.groupby('TIPO_DIA')
        if len(g) > 5
    ]
    
    if len(grupos) >= 2:
        stat, p = kruskal(*grupos)
        resultado_tipo_dia.append({
            'cd_loja': loja,
            'teste': 'TIPO_DIA',
            'p_valor': p
        })

resultado_tipo_dia = pd.DataFrame(resultado_tipo_dia)
resultado_tipo_dia

,cd_loja,teste,p_valor
0,03,TIPO_DIA,1.263948e-04
1,04,TIPO_DIA,7.703121e-07
2,05,TIPO_DIA,3.410819e-05
3,06,TIPO_DIA,2.054008e-21
4,07,TIPO_DIA,1.432152e-03


In [ ]:
#Kruskal-Wallis → “as distribuições são iguais?

resultado_dia_semana = []

for loja, df_loja in base.groupby('cd_loja'):
    grupos = [
        g['vendas_liquidas'].values
        for _, g in df_loja.groupby('DIA_SEMANA')
        if len(g) > 5
    ]
    
    if len(grupos) >= 2:
        stat, p = kruskal(*grupos)
        resultado_dia_semana.append({
            'cd_loja': loja,
            'teste': 'DIA_SEMANA',
            'p_valor': p
        })

resultado_dia_semana = pd.DataFrame(resultado_dia_semana)
resultado_dia_semana


,cd_loja,teste,p_valor
0,03,DIA_SEMANA,8.059375e-156
1,04,DIA_SEMANA,0.000000e+00
2,05,DIA_SEMANA,0.000000e+00
3,06,DIA_SEMANA,2.143437e-231
4,07,DIA_SEMANA,5.363395e-104


In [ ]:
#“as médias diferem?”

from scipy.stats import mannwhitneyu

resultados_promo = []

for loja in base['cd_loja'].unique():
    df = base[base['cd_loja'] == loja]

    g0 = df[df['em_promocao'] == 0]['vendas_liquidas']
    g1 = df[df['em_promocao'] == 1]['vendas_liquidas']

    # segurança mínima
    if len(g0) > 30 and len(g1) > 30:
        stat, p = mannwhitneyu(g0, g1, alternative='two-sided')

        resultados_promo.append({
            'cd_loja': loja,
            'teste': 'PROMOCAO',
            'p_valor': p,
            'media_sem_promo': g0.mean(),
            'media_com_promo': g1.mean(),
            'uplift_abs': g1.mean() - g0.mean(),
            'uplift_pct': (g1.mean() / g0.mean() - 1) * 100
        })

resultados_promo = pd.DataFrame(resultados_promo)
resultados_promo

,cd_loja,teste,p_valor,media_sem_promo,media_com_promo,uplift_abs,uplift_pct
0,03,PROMOCAO,0.0,3.636382,11.792358,8.155977,224.288249
1,04,PROMOCAO,0.0,5.263042,19.689513,14.426471,274.108976
2,05,PROMOCAO,0.0,5.487946,19.463788,13.975842,254.664335
3,06,PROMOCAO,0.0,6.723561,26.447825,19.724264,293.360366
4,07,PROMOCAO,0.0,5.471298,18.805912,13.334614,243.719398


In [ ]:
#pearman → “existe associação monotônica?”

from scipy.stats import spearmanr

resultado_preco = []

for loja, df_loja in base.groupby('cd_loja'):
    df_loja = df_loja.dropna(subset=['preco_gasolina', 'vendas_liquidas'])
    
    if len(df_loja) > 10:
        corr, p = spearmanr(
            df_loja['preco_gasolina'],
            df_loja['vendas_liquidas']
        )
        
        resultado_preco.append({
            'cd_loja': loja,
            'correlacao_spearman': corr,
            'p_valor': p
        })

resultado_preco = pd.DataFrame(resultado_preco)
resultado_preco

,cd_loja,correlacao_spearman,p_valor
0,03,0.013767,9.806046e-24
1,04,0.002612,1.372875e-02
2,05,0.003559,1.736784e-03
3,06,-0.073128,0.000000e+00
4,07,-0.026645,1.257397e-39


In [67]:
resumo = (
    resultado_tipo_dia
    .merge(resultado_dia_semana, on='cd_loja', how='outer', suffixes=('_tipo_dia', '_dia_semana'))
    .merge(resultado_preco, on='cd_loja', how='outer')
    .merge(resultados_promo, on='cd_loja', how='outer')
)

resumo

,cd_loja,teste_tipo_dia,p_valor_tipo_dia,teste_dia_semana,p_valor_dia_semana,correlacao_spearman,p_valor_x,teste,p_valor_y,media_sem_promo,media_com_promo,uplift_abs,uplift_pct
0,03,TIPO_DIA,0.000126,DIA_SEMANA,0.000000,0.013767,0.000000,PROMOCAO,0.000000,3.636382,11.792358,8.155977,224.288249
1,04,TIPO_DIA,0.000001,DIA_SEMANA,0.000000,0.002612,0.013729,PROMOCAO,0.000000,5.263042,19.689513,14.426471,274.108976
2,05,TIPO_DIA,0.000034,DIA_SEMANA,0.000000,0.003559,0.001737,PROMOCAO,0.000000,5.487946,19.463788,13.975842,254.664335
3,06,TIPO_DIA,0.000000,DIA_SEMANA,0.000000,-0.073128,0.000000,PROMOCAO,0.000000,6.723561,26.447825,19.724264,293.360366
4,07,TIPO_DIA,0.001432,DIA_SEMANA,0.000000,-0.026645,0.000000,PROMOCAO,0.000000,5.471298,18.805912,13.334614,243.719398


# random forest


In [73]:
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [74]:
data_corte = '2025-01-01'

features = [
    'preco_gasolina',
    'em_promocao',
    'TIPO_DIA',
    'DIA_SEMANA'
]

target = 'vendas_liquidas'

In [75]:
X_all = pd.get_dummies(
    base[features],
    columns=['TIPO_DIA', 'DIA_SEMANA'],
    drop_first=True
)

base_modelo = pd.concat(
    [
        base[['cd_loja', 'dt_emissao', target]],
        X_all
    ],
    axis=1
)

In [76]:
resultados_rf = []
importancias_rf = []

for loja in base_modelo['cd_loja'].unique():
    
    df_loja = base_modelo[base_modelo['cd_loja'] == loja]
    
    # split temporal
    mask_train = df_loja['dt_emissao'] < data_corte
    
    X = df_loja.drop(columns=['cd_loja', 'dt_emissao', target])
    y = df_loja[target]
    
    X_train = X[mask_train]
    X_test  = X[~mask_train]
    y_train = y[mask_train]
    y_test  = y[~mask_train]
    
    # proteção contra lojas pequenas
    if len(X_train) < 500 or len(X_test) < 50:
        continue
    
    rf = RandomForestRegressor(
        n_estimators=200,          # reduzido para velocidade
        max_depth=12,
        min_samples_leaf=50,
        random_state=42,
        n_jobs=-1
    )
    
    rf.fit(X_train, y_train)
    
    pred = rf.predict(X_test)
    
    mae = mean_absolute_error(y_test, pred)
    rmse = np.sqrt(mean_squared_error(y_test, pred))
    
    resultados_rf.append({
        'cd_loja': loja,
        'mae': mae,
        'rmse': rmse,
        'n_train': len(X_train),
        'n_test': len(X_test)
    })
    
    # feature importance
    imp = pd.DataFrame({
        'feature': X.columns,
        'importance': rf.feature_importances_,
        'cd_loja': loja
    })
    
    importancias_rf.append(imp)


In [77]:
resultados_rf = pd.DataFrame(resultados_rf)
resultados_rf.sort_values('rmse')


,cd_loja,mae,rmse,n_train,n_test
0,03,3.930308,11.744580,423605,123402
4,07,6.636008,16.875677,97122,161870
1,04,6.343706,20.105884,676545,235166
2,05,7.146954,21.628876,556985,238142
3,06,7.456064,23.052460,524998,193499


In [78]:
importancias_rf = pd.concat(importancias_rf)

(
    importancias_rf
    .groupby('feature')['importance']
    .mean()
    .sort_values(ascending=False)
    .head(10)
)

feature
em_promocao                0.607869
preco_gasolina             0.215079
DIA_SEMANA_Sábado          0.032498
TIPO_DIA_FERIAS            0.029323
DIA_SEMANA_Sexta-feira     0.027105
DIA_SEMANA_Quarta-feira    0.023737
DIA_SEMANA_Quinta-feira    0.018106
DIA_SEMANA_Terça-feira     0.013757
TIPO_DIA_POS FERIADO       0.011292
DIA_SEMANA_Segunda-feira   0.011252
Name: importance, dtype: float64

In [79]:
resultados_rf.head()

,cd_loja,mae,rmse,n_train,n_test
0,03,3.930308,11.744580,423605,123402
1,04,6.343706,20.105884,676545,235166
2,05,7.146954,21.628876,556985,238142
3,06,7.456064,23.052460,524998,193499
4,07,6.636008,16.875677,97122,161870


# forecast recursivo

In [80]:
base = base.sort_values(['cd_loja', 'dt_emissao'])

base['lag_7'] = (
    base
    .groupby('cd_loja')['vendas_liquidas']
    .shift(7)
)

base['media_7'] = (
    base
    .groupby('cd_loja')['vendas_liquidas']
    .shift(1)
    .rolling(7)
    .mean()
)

In [81]:
base_modelo = base.dropna().copy()

In [82]:
features = [
    'preco_gasolina',
    'em_promocao',
    'lag_7',
    'media_7',
    'TIPO_DIA',
    'DIA_SEMANA'
]

In [83]:
resultados = []
previsoes_30d = []

In [84]:
for loja in base_modelo['cd_loja'].unique():

    df_loja = base_modelo[base_modelo['cd_loja'] == loja].copy()

    # -----------------------------
    # Train / Test (time-based)
    # -----------------------------
    corte = pd.to_datetime('2025-01-01')

    train = df_loja[df_loja['dt_emissao'] < corte]
    test  = df_loja[df_loja['dt_emissao'] >= corte]

    X_train = train[features]
    y_train = train['vendas_liquidas']

    X_test  = test[features]
    y_test  = test['vendas_liquidas']

    # One-hot
    X_train = pd.get_dummies(X_train, columns=['TIPO_DIA', 'DIA_SEMANA'], drop_first=True)
    X_test  = pd.get_dummies(X_test, columns=['TIPO_DIA', 'DIA_SEMANA'], drop_first=True)

    X_test = X_test.reindex(columns=X_train.columns, fill_value=0)

    # -----------------------------
    # Modelo
    # -----------------------------
    rf = RandomForestRegressor(
        n_estimators=300,
        max_depth=12,
        min_samples_leaf=50,
        random_state=42,
        n_jobs=-1
    )

    rf.fit(X_train, y_train)

    # -----------------------------
    # Avaliação
    # -----------------------------
    pred_test = rf.predict(X_test)

    mae = mean_absolute_error(y_test, pred_test)
    rmse = np.sqrt(mean_squared_error(y_test, pred_test))

    resultados.append({
        'cd_loja': loja,
        'mae': mae,
        'rmse': rmse,
        'n_train': len(train),
        'n_test': len(test)
    })

    # =====================================================
    # 🔁 FORECAST RECURSIVO — 30 DIAS
    # =====================================================
    ultimos = df_loja.sort_values('dt_emissao').iloc[-7:].copy()
    historico = ultimos['vendas_liquidas'].tolist()

    ultima_data = df_loja['dt_emissao'].max()

    for passo in range(1, 31):

        data_prev = ultima_data + pd.Timedelta(days=passo)

        linha = {
            'preco_gasolina': df_loja['preco_gasolina'].iloc[-1],
            'em_promocao': 0,  # cenário base (sem promo)
            'lag_7': historico[-7],
            'media_7': np.mean(historico[-7:]),
            'TIPO_DIA': 'DIA COMUM',
            'DIA_SEMANA': data_prev.day_name(locale='pt_BR')
        }

        X_fut = pd.DataFrame([linha])
        X_fut = pd.get_dummies(X_fut, columns=['TIPO_DIA', 'DIA_SEMANA'], drop_first=True)
        X_fut = X_fut.reindex(columns=X_train.columns, fill_value=0)

        y_prev = rf.predict(X_fut)[0]

        historico.append(y_prev)

        previsoes_30d.append({
            'cd_loja': loja,
            'data': data_prev,
            'venda_prevista': y_prev
        })


In [85]:
df_resultados = pd.DataFrame(resultados)
df_resultados

,cd_loja,mae,rmse,n_train,n_test
0,03,3.896654,11.454542,420277,111867
1,04,6.108378,19.781031,676538,213655
2,05,6.547071,21.101554,556978,217383
3,06,7.169796,22.922237,524991,176400
4,07,6.133030,16.374157,97115,147203


In [86]:
df_forecast_30d = pd.DataFrame(previsoes_30d)
df_forecast_30d

,cd_loja,data,venda_prevista
0,03,2025-11-30,4.061187
1,03,2025-12-01,4.441212
2,03,2025-12-02,4.128555
3,03,2025-12-03,4.441212
4,03,2025-12-04,4.426200
...,...,...,...
145,07,2025-12-25,4.998339
146,07,2025-12-26,4.998339
147,07,2025-12-27,4.998339
148,07,2025-12-28,5.425706
